In [1]:
# Data Pipeline with pandas
import pandas as pd
from sodapy import Socrata
import pyodbc
import numpy as np
from sqlalchemy import create_engine

In [2]:
# Create SQLAlchemy engine for connect to SQL Server
engine = create_engine("mssql+pyodbc://<username>:<password>@<server_name>:1433/<database_name>?driver=ODBC+Driver+17+for+SQL+Server")

# Fetch NYC Data
client=Socrata("data.cityofnewyork.us","<APP_TOKEN>",username="<EMAIL>",password="<PASSWORD>")
results=client.get("43nn-pn8j",limit=1000000)

# Create dataframe from response
results_df=pd.DataFrame.from_records(results,index="camis")

# Replace nan values
results_df.replace({np.inf:np.nan,-np.inf:np.nan},inplace=True)

# Rename some columns, only to identify attributes
results_df.rename(columns={'dba':'name','building':'building_number','zipcode':'zip_code'},inplace=True)

# Delete nan and fill these with empty string
results_df=results_df[[
    'name',
    'building_number',
    'street',
    'zip_code',
    'phone',
    'cuisine_description',
    'latitude',
    'longitude',
    'community_board',
    'council_district',
    'census_tract',
    'bin',
    'bbl',
    'nta'
]].fillna("")

# Delete duplicates
results_df=results_df[~results_df.index.duplicated(keep='first')]

# Insert into SQL Server
results_df.to_sql('restaurant',con=engine,if_exists='append',index='id',chunksize=100)

-284